<a href="https://colab.research.google.com/github/prithvijaunjale/NLP/blob/master/text_summarization/text_summarization_kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gensim.models.keyedvectors as word2vec
import gc
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df = pd.read_csv('drive/My Drive/deep_learning/nlp/covid19/cord19_cleaned.csv', nrows=7000)
df.head()

,Unnamed: 0,paper_id,abstract,body_text
0,1,6979d40067c7917b6b92917ff8c3db8a440f5f4c,"Over the years, a vast array of information co...","Over the last decades, many host-pathogen inte..."
1,2,ed52c50fb3ff01140790f21c6e9716a8cd3c9fa8,KSHV achieves this by tethering its epigenome ...,Kaposi's sarcoma associated herpesvirus (KSHV)...
2,3,04421db3d34e7fd7c7486d66d21b34978a1fb362,Given the pleiotropic effects of type I IFNs (...,Type I IFNs provide a potent line of antiviral...
3,5,3c0c493a037dacafd2eda2e37518e0f0dff7955f,"We describe futures of ICU admission, demograp...","In 2009, cases of influenza like illness were ..."
4,6,2559cb0b4a2175212495ab7d44d9305db4a06c05,All positive-stranded RNA viruses with genomes...,Helicases and nucleic acid translocases are AT...


In [ ]:
def split_sentences(emails):
    """
    Splits the reviews into individual sentences
    """
    n_emails = len(emails)
    for i in range(n_emails):
        email = emails[i]
        #print(email)
        sentences = sent_tokenize(email)
        #print(sentences)
        for j in reversed(range(len(sentences))):
            sent = sentences[j]
            sentences[j] = sent.strip()
            if sent == '':
                sentences.pop(j)
        emails[i] = sentences

In [ ]:
rev_list = list(df['body_text'])
split_sentences(rev_list)
#Adding split reviews in the data frame
df['sent_tokens'] = rev_list
#Calculating lenght of sentences in each review
df['length_of_rv'] = df['sent_tokens'].map(lambda x: len(x))
df.head(3)

,Unnamed: 0,paper_id,abstract,body_text,sent_tokens,length_of_rv
0,1,6979d40067c7917b6b92917ff8c3db8a440f5f4c,"Over the years, a vast array of information co...","Over the last decades, many host-pathogen inte...","[Over the last decades, many host-pathogen int...",134
1,2,ed52c50fb3ff01140790f21c6e9716a8cd3c9fa8,KSHV achieves this by tethering its epigenome ...,Kaposi's sarcoma associated herpesvirus (KSHV)...,[Kaposi's sarcoma associated herpesvirus (KSHV...,307
2,3,04421db3d34e7fd7c7486d66d21b34978a1fb362,Given the pleiotropic effects of type I IFNs (...,Type I IFNs provide a potent line of antiviral...,[Type I IFNs provide a potent line of antivira...,155


In [ ]:
df.shape

(7000, 6)

In [ ]:
#Making vocabulary with reviews with max vocabs=5000. 
list_sentences_train = df['body_text']
max_features = 50000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
maxlen = 6000
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)

In [ ]:
# Pre trained Embeddings
embedder = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from tqdm import tqdm

# Calculating summaries for first 5000 reviews. 
how_many_summaries = 2
summary = [None] * how_many_summaries
for rv in tqdm(range(how_many_summaries)):
    review = df['sent_tokens'].iloc[rv]
    enc_email = embedder.encode(review)
    if(len(enc_email) > 0):
        n_clusters = int(np.ceil(len(enc_email)**0.5))
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        kmeans = kmeans.fit(enc_email)
        avg = []
        closest = []
        for j in range(n_clusters):
            idx = np.where(kmeans.labels_ == j)[0]
            #print("IDX is: ", idx)
            avg.append(np.mean(idx))
        closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_,\
                                                    enc_email)
        ordering = sorted(range(n_clusters), key=lambda k: avg[k])
        summary[rv] = ' '.join([review[closest[idx]] for idx in ordering])
        print("Done for review # = ", rv)
    else:
        print("This is not a valid review")



  0%|          | 0/2 [00:00<?, ?it/s]

 50%|█████     | 1/2 [00:22<00:22, 22.48s/it]

Done for review # =  0




100%|██████████| 2/2 [01:20<00:00, 40.25s/it]

Done for review # =  1


In [ ]:
df_2 = df.iloc[:2]
df_2['PredictedSummary'] = summary
df_2[['body_text', 'PredictedSummary']].to_csv('top_2_summary.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
string = df['abstract'][0]
from pprint import pprint
pprint(string)

('Over the years, a vast array of information concerning the interactions of '
 'viruses with their hosts has been collected. However, recent advances in '
 'proteomics and other system biology techniques suggest these interactions '
 'are far more complex than anticipated. One particularly interesting and '
 'novel aspect is the analysis of cellular proteins incorporated into mature '
 'virions. Though sometimes considered purification contaminants in the past, '
 'their repeated detection by different laboratories suggests that a number of '
 'these proteins are bona fide viral components, some of which likely '
 'contribute to the viral life cycles. The present mini review focuses on '
 'cellular proteins detected in herpesviruses. It highlights the common '
 'cellular functions of these proteins, their potential implications for '
 'host-pathogen interactions, discusses technical limitations, the need for '
 'complementing methods and probes potential future research avenues.')


In [ ]:
!pip install rouge-score

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(df['abstract'][0],summary[0])
scores

{'rouge1': Score(precision=0.18820224719101122, recall=0.5114503816793893, fmeasure=0.27515400410677615),
 'rougeL': Score(precision=0.08707865168539326, recall=0.2366412213740458, fmeasure=0.12731006160164274)}